In [1]:
# todo create pointing network!

In [120]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.nn import CrossEntropyLoss, NLLLoss

In [3]:
import transformers

/home/haryoaw/mambaforge/envs/sensei/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from transformers.models.bert.modeling_bert import BertLayer, BertConfig, BertAttention
from transformers import BertForTokenClassification

In [5]:
pointer_config = BertConfig(num_attention_heads=1, hidden_size=768, num_hidden_layers=2, intermediate_size=768)

In [5]:
# pointer network
tag_embedding = torch.nn.Embedding(12,5)
pos_embedding = torch.nn.Embedding(100,5)

# Linear SWISH GELU
linear = torch.nn.Linear(5,5)
swish = torch.nn.SiLU()
gelu = torch.nn.GELU()

# 2x encoder
encoder = BertLayer(pointer_config)

# Attention Layer! (single head)
attention = BertAttention(pointer_config)


In [6]:
import pandas as pd
from transformers import AutoTokenizer
import datasets

In [7]:
data_train = pd.read_csv("train_with_pointing.csv")
data_train = datasets.Dataset.from_pandas(data_train)

In [9]:
data_train

Dataset({
    features: ['informal', 'formal', 'point_indexes', 'label', 'len_label'],
    num_rows: 1922
})

In [27]:
tag_label_extract = data_train.to_pandas().point_labels.apply(lambda x: ' '.join([str(z) for z in x]))

In [28]:
pd.set_option("max_colwidth", 0)

In [29]:
data_train.to_pandas()[tag_label_extract.str.contains("86")]

,informal,formal,point_indexes,label,len_label,tag_labels,point_labels
1220,"sudah , harga barang xnumberx xnumberx xnumberx k , ongkir xnumberx xnumberx k , diskon xnumberx xnumberx k , biaya penanganan xnumberx , xnumberx k . seharusnya saya bayar xnumberx , xnumberx k saja , tapi malah ditagih xnumberx , xnumberx k .","sudah , harga barang xnumberx ribu , onkir xnumberx ribu , diskon xnumberx ribu , biaya penanganan xnumberx ribu . seharusnya saya bayar xnumberx ribu saja , tapi malah ditagih xnumberx ribu .","[1, 2, 3, 4, 5, 6, 7, 8, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 21, 22, 23, 24, 30, 0, 0, 0, 0, 0, 31, 32, 33, 34, 35, 41, 0, 0, 0, 0, 0, 42, 43, 44, 45, 46, 47, 54, 0, 0, 0, 0, 0, 0, 55, 56, 57, 58, 59, 60, 61, 68, 0, 0, 0, 0, 0, 0, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 85, 0, 0, 0, 0, 0, 0, 86, 0]","[2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 2, 2]",87,"[2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 3, 3, 3, 3, 3, 3, 2, 2]","[1, 2, 3, 4, 5, 6, 7, 8, 18, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 21, 22, 23, 24, 30, 0, 0, 0, 0, 0, 31, 32, 33, 34, 35, 41, 0, 0, 0, 0, 0, 42, 43, 44, 45, 46, 47, 54, 0, 0, 0, 0, 0, 0, 55, 56, 57, 58, 59, 60, 61, 68, 0, 0, 0, 0, 0, 0, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 85, 0, 0, 0, 0, 0, 0, 86, 0]"


In [100]:
data_train.to_pandas().len_label.value_counts()

len_label
20     97
13     93
14     92
16     91
19     88
11     84
21     82
17     80
15     80
12     74
18     73
22     70
25     64
10     62
28     61
31     59
9      56
23     56
26     52
24     51
27     50
29     47
32     42
30     40
35     36
33     35
8      27
34     27
37     26
36     22
38     16
39     11
40     11
41     8 
43     8 
47     5 
42     5 
48     5 
46     5 
54     4 
71     3 
52     2 
45     2 
88     2 
60     2 
44     2 
65     1 
49     1 
100    1 
51     1 
84     1 
59     1 
73     1 
50     1 
55     1 
93     1 
82     1 
7      1 
57     1 
76     1 
Name: count, dtype: int64

In [12]:
def create_label_dict(max_mask=3, use_pointing=True):
    label_map = {'PAD': 0, 'SWAP': 1, 'KEEP': 2, 'DELETE': 3}
    # Create Insert 1 MASK to insertion N MASKS.
    for i in range(1, max_mask+1):
        label_map[f'KEEP|{i}'] = len(label_map)
    if not use_pointing:
        label_map[f'DELETE|{i}'] = len(label_map)
    return label_map

In [25]:
agg = []
for x in data_train.to_pandas().point_labels:
    agg.extend(x)
set(agg)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 85,
 86}

In [77]:
99 in agg

True

In [17]:
label_dict = create_label_dict(25)

In [21]:
from mbee import compute_edits_and_insertions
from insert_convert import InsertionConverter, get_number_of_masks
from trying import PointingConverter

In [22]:
point_converter = PointingConverter({}, False)

In [43]:
def tokenize_function_src_tgt(examples, tokenizer, src="informal", tgt="formal"):
    returned_dict = {f"{src}_{i}": j for i,j in tokenizer(examples[src]).items()}
    returned_dict.update({f"{tgt}_{i}": j for i,j in tokenizer(examples[tgt]).items()})
    return returned_dict

In [44]:
data_train = data_train.map(
    tokenize_function_src_tgt,
    batched=True,
    fn_kwargs={
        "tokenizer": AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
    },
)

Map: 100%|██████████| 1922/1922 [00:00<00:00, 14200.00 examples/s]


In [45]:
data_train

Dataset({
    features: ['informal', 'formal', 'point_indexes', 'label', 'len_label', 'tag_labels', 'point_labels', 'informal_input_ids', 'informal_token_type_ids', 'informal_attention_mask', 'formal_input_ids', 'formal_token_type_ids', 'formal_attention_mask'],
    num_rows: 1922
})

In [46]:
def create_pointer_labels(points, label_map):
    labels = [t.added_phrase for t in points]
    point_indexes = [t.point_index for t in points]
    point_indexes_set = set(point_indexes)
    new_labels = []
    for i, added_phrase in enumerate(labels):
        if i not in point_indexes_set:
            new_labels.append(label_map["DELETE"])
        elif not added_phrase:
            new_labels.append(label_map["KEEP"])
        else:
            new_labels.append(label_map["KEEP|" + str(len(added_phrase.split()))])
    return new_labels

In [47]:
def generate_tokenized(examples, tokenizer, label_dict, point_converter, src="informal", tgt="formal"):
    src_tokenized = tokenizer.tokenize(examples[src], add_special_tokens=True)
    tgt_tokenized = tokenizer.tokenize(examples[tgt], add_special_tokens=True)
    points = point_converter.compute_points(src_tokenized, ' '.join(tgt_tokenized))
    label = create_pointer_labels(points, label_dict)
    point_indexes = [t.point_index for t in points] 
    # change them to torch tensors
    label = label
    point_indexes = point_indexes
    return {f"tag_labels": label, f"point_labels": point_indexes}

In [48]:
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")

In [49]:
data_train = data_train.map(
    generate_tokenized,
    batched=False,
    fn_kwargs={
        "tokenizer": tokenizer,
        "label_dict": label_dict,
        "point_converter": point_converter
    },
)

Map:   0%|          | 0/1922 [00:00<?, ? examples/s]

Map: 100%|██████████| 1922/1922 [00:00<00:00, 2160.49 examples/s]


In [50]:
data_train

Dataset({
    features: ['informal', 'formal', 'point_indexes', 'label', 'len_label', 'tag_labels', 'point_labels', 'informal_input_ids', 'informal_token_type_ids', 'informal_attention_mask', 'formal_input_ids', 'formal_token_type_ids', 'formal_attention_mask'],
    num_rows: 1922
})

In [51]:
len(data_train['point_labels'][0])

29

In [52]:
len(data_train['formal_input_ids'][0])

27

In [53]:
data_train

Dataset({
    features: ['informal', 'formal', 'point_indexes', 'label', 'len_label', 'tag_labels', 'point_labels', 'informal_input_ids', 'informal_token_type_ids', 'informal_attention_mask', 'formal_input_ids', 'formal_token_type_ids', 'formal_attention_mask'],
    num_rows: 1922
})

In [54]:
bert_koto = BertForTokenClassification.from_pretrained("indolem/indobert-base-uncased", num_labels=12)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
# collator


from copy import deepcopy


class FelixCollator:
    def __init__(self, tokenizer, pad_label=-100, pad_label_as_input=80):
        # change the pad_label_as_input and point_label_as_input
        # these must be the len of the label vocab and point_label vocab
        self.tokenizer = tokenizer
        self.pad_label = pad_label
        self.pad_label_as_input = pad_label_as_input

    def __call__(self, batch):
        # batch is a list of dicts
        output_dict = {}
        informal_input_ids, informal_attention_mask = [
            [i[col] for i in batch]
            for col in [
                "informal_input_ids",
                "informal_attention_mask",
            ]
        ]
        formal_input_ids = [i["formal_input_ids"] for i in batch]

        tag_label = [i["tag_labels"] for i in batch]

        tokenized_output = self.tokenizer.pad(
            {
                "input_ids": informal_input_ids,
                "attention_mask": informal_attention_mask,
            },
            return_tensors="pt",
        )
        tokenized_output["token_type_ids"] = torch.zeros_like(
            tokenized_output["input_ids"]
        )

        output_label = self.tokenizer.pad(
            {
                "input_ids": formal_input_ids,
            },
            return_tensors="pt",
        )
        out_label = output_label["input_ids"]
        # change pad token to -100
        out_label[out_label == self.tokenizer.pad_token_id] = self.pad_label
        # print(out_label)
        output_dict.update(tokenized_output)
        output_dict['labels'] = out_label
        # print(output_dict)
        # add tag_label to output_dict
        # each tag_label is a list of labels (list) with different length
        # pad first
        max_len = max([len(i) for i in tag_label])
        tag_label = [i + [self.pad_label] * (max_len - len(i)) for i in tag_label]
        tag_label = torch.tensor(tag_label)

        output_dict["tag_labels"] = tag_label
        output_dict['tag_labels_input'] = deepcopy(tag_label)
        output_dict['tag_labels_input'][output_dict['tag_labels_input'] == self.pad_label] = self.pad_label_as_input

        # add point_label to output_dict
        # same as above
        point_label = [i["point_labels"] for i in batch]
        max_len = max([len(i) for i in point_label])
        point_label = [i + [self.pad_label] * (max_len - len(i)) for i in point_label]
        point_label = torch.tensor(point_label)

        output_dict["point_labels"] = point_label

        return output_dict

In [93]:
from torch.utils.data import DataLoader

loader = DataLoader(data_train, batch_size=2, collate_fn=FelixCollator(tokenizer, pad_label_as_input=len(label_dict)))

In [94]:

for current_batch in loader:
    print(current_batch.keys())
    input_to_koto = {k: v for k, v in current_batch.items() if k in ['input_ids', 'attention_mask', 'token_type_ids']}
    print(current_batch['labels'].shape)
    for x in ['token_type_ids', 'input_ids', 'attention_mask']:
        print(input_to_koto[x].shape, x)
    tag_pred = bert_koto(**input_to_koto, labels=current_batch['tag_labels'])
    break

dict_keys(['input_ids', 'attention_mask', 'token_type_ids', 'labels', 'tag_labels', 'tag_labels_input', 'point_labels'])
torch.Size([2, 29])
torch.Size([2, 29]) token_type_ids
torch.Size([2, 29]) input_ids
torch.Size([2, 29]) attention_mask


In [95]:
label_dict

{'PAD': 0,
 'SWAP': 1,
 'KEEP': 2,
 'DELETE': 3,
 'KEEP|1': 4,
 'KEEP|2': 5,
 'KEEP|3': 6,
 'KEEP|4': 7,
 'KEEP|5': 8,
 'KEEP|6': 9,
 'KEEP|7': 10,
 'KEEP|8': 11,
 'KEEP|9': 12,
 'KEEP|10': 13,
 'KEEP|11': 14,
 'KEEP|12': 15,
 'KEEP|13': 16,
 'KEEP|14': 17,
 'KEEP|15': 18,
 'KEEP|16': 19,
 'KEEP|17': 20,
 'KEEP|18': 21,
 'KEEP|19': 22,
 'KEEP|20': 23,
 'KEEP|21': 24,
 'KEEP|22': 25,
 'KEEP|23': 26,
 'KEEP|24': 27,
 'KEEP|25': 28}

In [75]:
current_batch['tag_labels']

tensor([[   2,    4,    3,    3,    3,    2,    2,    2,    2,    2,    2,    2,
            2,    5,    3,    3,    3,    2,    3,    5,    3,    3,    5,    3,
            2,    2,    2,    4,    2],
        [   2,    2,    4,    3,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    4,    2, -100]])

In [67]:
current_batch = next(iter(loader))

input_to_koto = {k: v for k, v in current_batch.items() if k in ['input_ids', 'attention_mask', 'token_type_ids']}
input_to_koto['labels'] = current_batch['tag_labels']
bert_koto(**input_to_koto)

TokenClassifierOutput(loss=tensor(2.4634, grad_fn=<NllLossBackward0>), logits=tensor([[[-0.2959,  0.3518,  0.3980, -0.0159, -0.4651,  0.5296, -0.7702,
          -0.1949,  0.6155,  0.3015,  0.0242, -0.6207],
         [ 0.0052,  0.6299, -0.1825, -0.3160, -0.1635,  0.2809, -0.6057,
           0.3321,  0.7368,  0.5562, -0.1229, -0.7043],
         [ 0.5985, -0.8385,  0.3260, -0.3261,  0.1031,  0.5229, -0.0964,
          -0.0522,  0.4766, -0.5926, -0.7479, -0.2680],
         [ 0.5692, -0.1968, -0.1931, -0.2463,  0.1139,  0.3532,  0.2381,
          -0.0285,  0.0910, -1.0697, -0.7346, -0.2566],
         [ 0.3286,  0.3003,  0.0310,  0.4080, -0.2227,  0.0400, -0.5407,
          -0.5792, -0.5444, -0.3461, -0.5294, -0.4473],
         [ 0.8533, -0.2651, -0.0610,  0.0383,  0.7926,  0.1733, -1.0643,
           0.5748, -0.0090,  0.3621,  0.1694, -0.2701],
         [-0.3819, -0.3728, -0.1506, -0.4952, -0.8633,  0.0314, -0.2889,
           0.0681, -0.3612,  0.3071, -0.0326, -0.3551],
         [-0.0512, 

In [69]:
next(iter(loader))

{'input_ids': tensor([[    3, 11450,  1862,   932,   945, 10121,    66,  8014,  1604,   962,
           1843,  2587,  4207,   933, 17849, 17104, 21463, 12411,  1476,    16,
          14099, 17849,    18, 21140, 18070,  6359,   962, 10155,     4],
         [    3,  4863,  6097,  2118,    18,  1731,  2882,  3888,  6505,  1881,
          10896,  4362,    16,  1925,  2643,  6813,     6,  2022,  4942,  1560,
           2289,     6,  9153, 23120,    18,  5218,  3774,     4,     0]]),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0, 0, 0]]),
 'labels': tensor([[    3, 11450,  1818, 1012

In [58]:
from transformers.models.bert.modeling_bert import BertLayer, BertConfig, BertAttention
from transformers import BertForTokenClassification

In [59]:
from torch import nn

In [60]:
from transformers import BertModel
from transformers.models.bert.modeling_bert import BertSelfAttention


In [155]:
class PointerNetwork(nn.Module):
    def __init__(self, pointer_config) -> None:
        super().__init__()
        self.pointer_config = pointer_config
        self.bert = BertModel(pointer_config)
        self.last_attention = BertSelfAttention(pointer_config)
        self.nll_loss = NLLLoss()

    def forward(self, input_ids, attention_mask, token_type_ids, labels=None):
        bert_output = self.bert(input_ids, attention_mask, token_type_ids)

        input_shape = input_ids.size()
        extended_attention_mask: torch.Tensor = self.bert.get_extended_attention_mask(
            attention_mask, input_shape
        )
        # print(bert_output[0].shape)
        _, last_attention = self.last_attention(
            bert_output[0], extended_attention_mask, output_attentions=True
        )
        if labels is not None:
            loss = self.nll_loss(last_attention.view(-1, last_attention.shape[-1]), labels.view(-1))
            return loss, last_attention
        return None, last_attention

In [156]:
pointer_network_config = BertConfig(
    vocab_size=len(label_dict) + 1,
    num_hidden_layers=2,
    num_attention_heads=1,
    pad_token_id=len(label_dict),
)  # + 1 as the pad token

In [157]:
pointer_network = PointerNetwork(pointer_network_config)

In [158]:
input_to_pointer = next(iter(loader))

In [159]:
input_to_pointer.keys()

dict_keys(['input_ids', 'attention_mask', 'token_type_ids', 'labels', 'tag_labels', 'tag_labels_input', 'point_labels'])

In [160]:
input_to_pointer_real = {
    k: v for k, v in input_to_pointer.items() if k in ['tag_labels_input', 'attention_mask', 'token_type_ids']
}
input_to_pointer_real['input_ids'] = input_to_pointer_real.pop('tag_labels_input')

In [161]:
input_to_pointer_real['input_ids'] = input_to_pointer_real.pop('tag_labels_input')

In [167]:
softmax_out = pointer_network(**input_to_pointer_real)
# make it as log softmax
softmax_out = torch.nn.LogSoftmax(dim=-1)(softmax_out)

In [168]:
NLLLoss()(softmax_out.view(-1, softmax_out.shape[-1]), input_to_pointer['point_labels'].view(-1))

tensor(3.3638, grad_fn=<NllLossBackward0>)

In [34]:
# pointer network
tag_embedding = torch.nn.Embedding(12,5)
pos_embedding = torch.nn.Embedding(100,5)

# Linear SWISH GELU
linear = torch.nn.Linear(5,5)
swish = torch.nn.SiLU()
gelu = torch.nn.GELU()

# 2x encoder
encoder = BertLayer(pointer_config)

# Attention Layer! (single head)
attention = BertAttention(pointer_config)
